# Totais de matrículas - Série Histórica

Pode ser que seu interesse, ao trabalhar com os microdados das matrículas, seja selecionar determinada série histórica.

Os arquivos utilizados podem ser acessados no [Portal de Dados Abertos](http://dados.prefeitura.sp.gov.br/dataset/microdados-da-rede-municipal-de-ensino-matriculas). 

É possível que você necessite de só alguns dos anos, ou um intervalo menor. Por isso, nem sempre é interessante trabalhar com a base completa -- as operações podem ficar muito lentas ou mesmo inviáveis, por insuficiência de memória.

[Saiba mais](https://github.com/prefeiturasp/dados-educacao/blob/master/README.md) sobre os tutoriais dos dados abertos da SME.

*Importante: também estamos aprendendo! Tem sugestões para melhorar os códigos? Não deixe de contribuir com este repositório :)*

In [1]:
import pandas as pd
import numpy as np

Trabalhamos aqui com os arquivos .csv disponíveis no Portal de Dados Abertos, já descompactados. Diferente do outro tutorial, porém, importaremos apenas algumas colunas (com que vamos trabalhar), para evitar insuficiência de memória: 

In [2]:
anos = ['2009','2010', '2011', '2012', '2013', '2014', '2015', '2016']
#anos letivos que serão selecionados

colunas = ['AN_LETIVO', 'CD_ALUNO_SME', 'SITUACAO_MAT', 'MODALIDADE_SEGMENTO']
#variáveis que serão selecionadas

Um pouquinho de paciência para rodar o código abaixo (em especial com a quantidade de anos acima!)

In [3]:
df_mat_total = []

for ano in anos:
    
    matriculas = pd.read_csv('Microdados_EOL_Matriculas_'+str(ano)+'.csv', sep=';', encoding='ISO-8859-1', low_memory=False, usecols=colunas)
    #Lê o arquivo e transforma em DataFrame
    
    df_mat_total.append(matriculas)

df_mat_total = pd.concat(df_mat_total, axis=0, ignore_index=True)

In [40]:
filtros = ['CRECHE', 'PRE', 'Fund1', 'Fund2']
#escolhendo algumas modalidades/segmentos específicos para totalizar

df_filtrada = df_mat_total[(df_mat_total.MODALIDADE_SEGMENTO.isin(filtros)) 
                           & (df_mat_total['SITUACAO_MAT'] == 'Concluido')]

In [41]:
df_filtrada.tail()

,AN_LETIVO,MODALIDADE_SEGMENTO,CD_ALUNO_SME,SITUACAO_MAT
13917086,2016,Fund2,4203276,Concluido
13917087,2016,Fund2,4846283,Concluido
13917088,2016,Fund2,4986111,Concluido
13917089,2016,Fund2,3597690,Concluido
13917090,2016,Fund1,5972658,Concluido


In [38]:
df_filtrada.shape

(7014157, 4)

In [50]:
df_filtrada['MODALIDADE_SEGMENTO'] = df_filtrada['MODALIDADE_SEGMENTO'].astype('category', ordered=True, categories=['CRECHE', 'PRE', 'Fund1', 'Fund2']).cat.rename_categories(['Ed. Infantil - Creche', 'Ed. Infantil - Pré', 'Fundamental - Anos Iniciais', 'Fundamental - Anos Finais'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
totais = pd.pivot_table(df_filtrada, values='CD_ALUNO_SME',
                        index='AN_LETIVO',
                        columns='MODALIDADE_SEGMENTO',
                        aggfunc=np.count_nonzero)

In [52]:
totais

MODALIDADE_SEGMENTO,Ed. Infantil - Creche,Ed. Infantil - Pré,Fundamental - Anos Iniciais,Fundamental - Anos Finais
AN_LETIVO,,,,
2009,123263,310051,246280,243675
2010,130378,289531,233269,246612
2011,195652,185398,215088,242796
2012,213561,182327,209006,239943
2013,214452,200394,205071,237507
2014,228311,202558,246661,177323
2015,260776,204573,245089,170289
2016,284115,215123,245641,169444


In [53]:
totais.to_excel('totais.xlsx')